In [ ]:
import pandas as pd
import numpy as np
import matplotlib
import plotly.express as px
import matplotlib.pyplot as plt
from collections import namedtuple

In [ ]:
%run ../modules/utils.ipynb
%run ../modules/cds.ipynb
%run ../modules/preprocessing.ipynb

In [ ]:
# df = pd.read_csv('/Users/bryan/Documents/SeqDataSets/Ning1024/12mixtures/19-Table 1.csv')
# df = pd.read_csv('/Users/bryan/Documents/SeqDataSets/Ning1024/12mixtures/30-Table 1.csv')
# df = pd.read_csv('/Users/bryan/Studio/lcmsseq/examples/compounds.csv', skiprows=2)
# df = pd.read_excel('/Users/bryan/Documents/SeqDataSets/Ning1024/5mix_MFE.xlsx')
# df = pd.read_excel('/Users/bryan/Documents/SeqDataSets/Ning1024/2mix_180521s14_100.xls')
# df = pd.read_csv('/Users/bryan/Documents/SeqDataSets/Ning1024/12mixtures/12mix.csv')
# df = pd.read_excel('/Users/bryan/Documents/BioPharmaFinder/ZhangLab/Data/Excels/tRNA_Phe_0724_std.xlsx')
# df = pd.read_excel('/Users/bryan/Documents/BioPharmaFinder/ZhangLab/oligosdegradsample2-1.xlsx')
# df = pd.read_excel('/Users/bryan/Documents/BioPharmaFinder/ZhangLab/Data/Excels/total_tRNA_acid_degradated_20to40_component_only.xlsx')
# df = pd.read_excel('/Users/bryan/Documents/BioPharmaFinder/ZhangLab/Data/Excels/total_tRNA_acid_deg.xlsx')
# df = thermo_df(df)
# df = df[df.Mass < 20000]
# df = pd.read_excel('/Users/bryan/Documents/SeqDataSets/PaperData/ACS/032919s07.xls')
df = pd.read_excel('/Users/bryan/Documents/SeqDataSets/PaperData/ACS/111418s05.xls')
# df = pd.read_excel('/Users/bryan/Documents/SeqDataSets/PaperData/ACS/100918s06.xls')
df.shape

In [ ]:
plot_zone(df)

In [ ]:

idxs = list()
# create our callback function
def update_point(trace, points, selector):
    print('points {}'.format(points.point_inds) )
    idxs.extend(points.point_inds)
f = zone_selection(df, update_point)
f

In [ ]:
df_chosen = df.iloc[idxs]
df_chosen.shape

In [ ]:
bcr_5p = base_calling_random(df_chosen)
plt, _ = plot_basecalling(*bcr_5p, annotate=False)

In [ ]:
def process_slopes(df, mass_pairs, radius=0.25, median=-1):
    slopes = list()
    for t in mass_pairs:
        df_pair = df[df.Mass.isin(t)]
        if df_pair.empty:
            continue
        
        slope = 1E3 * (df_pair.iloc[0].RT - df_pair.iloc[1].RT)/(df_pair.iloc[0].Mass - df_pair.iloc[1].Mass)
        if 0. < slope < float('inf'):
            slopes.append(slope)
    if median < 0:
        median = np.median(slopes)
    lo = median - radius
    lo = lo if lo > 0 else 0
    hi = median + radius
#     median = slope_range(slopes, radius * 2)
#     if not median:
#         return None
#     lo = median.lo
#     hi = median.hi
#     ranges = slope_ranges(slopes)
        
    count = 0
    count_ignore = 0
    accepted_pairs = list()
    for t in mass_pairs:
        df_pair = df[df.Mass.isin(t)]
        if df_pair.empty:
            continue
        
        slope = 1E3 * (df_pair.iloc[0].RT - df_pair.iloc[1].RT)/(df_pair.iloc[0].Mass - df_pair.iloc[1].Mass)
#         if slope > 0.:
        if lo < slope < hi:
#         if any([i[0] < slope < i[1] for i in ranges]):
            slopes.append(slope)
#             plt.plot(df_pair.Mass, df_pair.RT, 'green')
            count += 1
            accepted_pairs.append(df_pair)
        else:
            count_ignore += 1
    
#     print('median {:.2f} lo-hi {:.3f}-{:.3f} accept {} reject {}'.format((lo+hi)/2, lo, hi, count, count_ignore))
    print('accept {} reject {}'.format(count, count_ignore))
    return accepted_pairs

In [ ]:
def slope_ranges(l, ratio=0.5, top=2):
    print(l)
    hist = np.histogram(l, bins='auto')
    max_bin_count = max(hist[0])
    loc_counts = list(enumerate(hist[0]))
    top_loc_counts = [i for i in loc_counts if i[1] > max_bin_count * ratio]
    top_loc_counts = sorted(top_loc_counts, key=lambda x: x[1], reverse=True)[:top]
    ranges = [(hist[1][i[0]], hist[1][i[0]+1]) for i in top_loc_counts]
    print(ranges)
    return ranges

In [ ]:
win_pos = 200
win_size = 600
TOP = 2
pairs = list()
while win_pos + win_size < df.Mass.max():
    df_win = df_chosen[(df_chosen.Mass > win_pos) & (df_chosen.Mass < win_pos + win_size)]
#     print(df_win.shape)
    if df_win.shape[0] > 1:
#         continue
        print('win {}-{}'.format(win_pos, win_pos+win_size))
        slope, _ = np.polyfit(df_win.Mass, df_win.RT, 1)
        slope = slope * 1E3
        print(slope)
        bcr_5p = base_calling_random(df_win, silence=True)
        accepted_pairs = process_slopes(*bcr_5p, radius=0.40, median=-1)
        if accepted_pairs:
            pairs.extend(accepted_pairs)
    win_pos += 100

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(16, 12))
plt.xlabel('Monoisotopic Mass (Da)', fontname="Arial", fontsize=15, color='black')
plt.ylabel('Retention Time (min)', fontname="Arial", fontsize=15, color='black')
plt.xticks(fontname="Arial", size=13, color='black')
plt.yticks(fontname="Arial", size=13, color='black')
plt.scatter(df.Mass, df.RT)
for pair in pairs:
    plt.plot(pair.Mass, pair.RT, color='g')
plt.savefig('/Users/bryan/Downloads/3mix.png', transparent=True)
# plt.savefig('/Users/bryan/Downloads/total_line_only.png', transparent=True)

In [ ]:
df_all = pd.concat(pairs)
df_all.shape
df_all.drop_duplicates(inplace=True)
df_all.shape

In [ ]:
bcr = base_calling_random(df_all)
plt, _ = plot_basecalling(*bcr, annotate=False)

In [ ]:
bcr[0].to_excel('/Users/bryan/Downloads/total_sliding_win.xlsx')

In [ ]:
win_pos = 200
win_size = 600
TOP = 2
pairs = list()
while win_pos + win_size < df_all.Mass.max():
    df_win = df_all[(df_all.Mass > win_pos) & (df_all.Mass < win_pos + win_size)]
#     print(df_win.shape)
    if df_win.shape[0] > 1:
#         continue
        print('win {}-{}'.format(win_pos, win_pos+win_size))
        slope, _ = np.polyfit(df_win.Mass, df_win.RT, 1)
        slope = slope * 1E3
        print(slope)
        bcr_5p = base_calling_random(df_win, silence=True)
        accepted_pairs = process_slopes(*bcr_5p, radius=0.40, median=-1)
        if accepted_pairs:
            pairs.extend(accepted_pairs)
    win_pos += 100

import matplotlib.pyplot as plt
plt.figure(figsize=(16, 12))
plt.xlabel('Monoisotopic Mass (Da)', fontname="Arial", fontsize=15, color='black')
plt.ylabel('Retention Time (min)', fontname="Arial", fontsize=15, color='black')
plt.xticks(fontname="Arial", size=13, color='black')
plt.yticks(fontname="Arial", size=13, color='black')
plt.scatter(df.Mass, df.RT)
for pair in pairs:
    plt.plot(pair.Mass, pair.RT, color='g')
# plt.savefig('/Users/bryan/Downloads/total.png', transparent=True)
# plt.savefig('/Users/bryan/Downloads/total_line_only.png', transparent=True)

In [ ]:

idxs = list()
# create our callback function
def update_point(trace, points, selector):
    print('points {}'.format(points.point_inds) )
    idxs.extend(points.point_inds)
f = zone_selection(df_all, update_point)
f

In [ ]:
df_all_chosen = df_all.iloc[idxs]
df_all_chosen.shape

In [ ]:
bcr = base_calling_random(df_all_chosen)
plt, _ = plot_basecalling(*bcr, annotate=False)

# Utils Functions

In [ ]:
def plot_basecalling3(df, mass_pairs, endpoints=pd.DataFrame(), annotate=True, plt=None, median=-1):
    if not plt:
        plt = matplotlib.pyplot
    fig = plt.figure(figsize=(16, 12))
    plt.xlabel('Monoisotopic Mass (Da)', fontname="Arial", fontsize=15, color='black')
    plt.ylabel('Retention Time (min)', fontname="Arial", fontsize=15, color='black')
    plt.xticks(fontname="Arial", size=13, color='black')
    plt.yticks(fontname="Arial", size=13, color='black')
        
    plt.scatter(df.Mass, df.RT)

    # calc slopes
    slopes = list()
    for t in mass_pairs:
        df_pair = df[df.Mass.isin(t)]
        if df_pair.empty:
            continue
        
        slope = 1E3 * (df_pair.iloc[0].RT - df_pair.iloc[1].RT)/(df_pair.iloc[0].Mass - df_pair.iloc[1].Mass)
        if slope > 0.:
            slopes.append(slope)
    if median < 0:
        median = np.median(slopes)
    radius = 0.4
    lo = median - radius
    lo = lo if lo > 0 else 0
    hi = median + radius
#     median = slope_range(slopes, 0.5)
#     if not median:
#         return plt, fig
#     lo = median.lo
#     hi = median.hi
        
    count = 0
    count_ignore = 0
    for t in mass_pairs:
        df_pair = df[df.Mass.isin(t)]
        if df_pair.empty:
            continue
        
        slope = 1E3 * (df_pair.iloc[0].RT - df_pair.iloc[1].RT)/(df_pair.iloc[0].Mass - df_pair.iloc[1].Mass)
#         if slope > 0.:
        if lo < slope < hi:
#             slopes.append(slope)
            plt.plot(df_pair.Mass, df_pair.RT, 'green')
            count += 1
        else:
            count_ignore += 1
        
        if not annotate:
            continue
            
        idmax = df_pair['Mass'].idxmax()
        plt.annotate(s=t[2], size=15, xy=(df_pair.loc[idmax].Mass, df_pair.loc[idmax].RT), 
                     textcoords="offset points", xytext=(-10, 10), ha='center', color='black')
        
        mass = '{:.2f}'.format(df_pair.loc[idmax].Mass)
        plt.annotate(s=mass, size=13, xy=(df_pair.loc[idmax].Mass, df_pair.loc[idmax].RT), 
                     textcoords="offset points", xytext=(10, -20), ha='center')

    print('median {:.2f} lo-hi {:.3f}-{:.3f} accept {} reject {}'.format((lo+hi)/2, lo, hi, count, count_ignore))
    print(slopes)
    if not endpoints.empty:
        plt.scatter(endpoints.Mass, endpoints.RT, color='r')
        print(endpoints[['Mass', 'RT', 'Vol']])
    return plt, fig

# Single Window

In [ ]:
win_pos = 2100
win_size = 600

In [ ]:
df_win = df_chosen[(df_chosen.Mass > win_pos) & (df_chosen.Mass < win_pos + win_size)]
df_win.shape

In [ ]:
plt = plot_zones(df_chosen, df_win)

In [ ]:
plt = plot_zone(df_win, trend=True)
slope, _ = np.polyfit(df_win.Mass, df_win.RT, 1)
slope = slope * 1E3
print(slope)

In [ ]:
def calc_slope(df, mass_pairs):
    masses = list(set(sum(mass_pairs, ())))
    df_basecalls = df[df.Mass.isin(masses)]
    slope, _ = np.polyfit(df_basecalls.Mass, df_basecalls.RT, 1)
    slope = slope * 1E3
    print(slope)
    return df_basecalls, slope

In [ ]:
bcr_5p = base_calling_random(df_win, silence=True)
df_bc, slope = calc_slope(*bcr_5p)
# plt, _ = plot_basecalling(*bcr_5p, annotate=False)
plt, _ = plot_basecalling3(*bcr_5p, annotate=False, median=slope)
plt.scatter(df_win.Mass, df_win.RT, color='C0')
plt.savefig('/Users/bryan/Downloads/win.png', transparent=True)

In [ ]:
plot_zone(df_bc, trend=True)
slope, _ = np.polyfit(df_bc.Mass, df_bc.RT, 1)
print(slope * 1E3)

In [ ]:
# fig = px.scatter(x=df_win.Mass, y=df_win.RT)
# fig.show()